# Problem Statement: **AtliQ's Social Media Monitoring for COVID-19 Insights using BERT**

### AtliQ is launching a social media monitoring system to identify tweets related to COVID-19, aiming to track trends, misinformation, and public sentiment. Your task is to fine-tune a pre-trained BERT model using PyTorch to classify whether a given tweet is about COVID-19 or not.

**References:**

1. **Attention is All you Need:** [Click Here](https://arxiv.org/abs/1706.03762)

2. **BERT:** [Click Here](https://arxiv.org/abs/1810.04805)



---



Imports and CUDA

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from transformers import BertForSequenceClassification
from transformers import AutoTokenizer

# Check if CUDA (GPU) is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

/Users/saurabhsule/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


**Step1:** Dataset Overview

* We use **covid_twitter_dataset_codebasics_DL** dataset which has 5287 rows.
* Columns: ID, text and target
* The feature column (text) contains the tweet content, and the target column (target) contains the binary labels indicating whether a tweet is COVID-19-related (**1**) or not (**0**).


In [2]:
data = pd.read_csv('covid_twitter_dataset_codebasics_DL.csv')
data.head()

,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,266 Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5287 entries, 0 to 5286
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      5287 non-null   object
 1   text    5287 non-null   object
 2   target  5287 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 124.0+ KB




---



**Step2:** Split the dataset

* Train:Test :: 70:30



---



In [4]:
train_X, test_X, train_Y, test_Y = train_test_split(data['text'], data['target'], train_size = 0.7, random_state = 42)

**Step3:** Tokenization

* The AutoTokenizer from the Hugging Face library is used to load the pre-trained BERT tokenizer (bert-base-cased).


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [6]:
train_tokens = tokenizer(list(train_X), padding = True, truncation=True)
test_tokens = tokenizer(list(test_X), padding = True, truncation=True)

In [11]:
train_tokens.keys()

KeysView({'input_ids': [[101, 3725, 9216, 8653, 2740, 124, 5187, 1559, 5465, 1545, 1367, 5966, 1580, 9493, 2740, 122, 5539, 1571, 26516, 4859, 1545, 4735, 1116, 21606, 5117, 1477, 3993, 1604, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1177, 3587, 1149, 1139, 5855, 8186, 189, 1151, 1313, 1290, 1523, 1314, 1480, 1105, 1195, 1169, 189, 1525, 1140, 5456, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 7911, 1184, 15078, 6778, 2947, 1113, 1111, 2277, 9913, 3025, 1115, 1120, 3047, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
print(train_tokens['input_ids'][0])
print(tokenizer.decode(train_tokens['input_ids'][0]))

[101, 3725, 9216, 8653, 2740, 124, 5187, 1559, 5465, 1545, 1367, 5966, 1580, 9493, 2740, 122, 5539, 1571, 26516, 4859, 1545, 4735, 1116, 21606, 5117, 1477, 3993, 1604, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[CLS] Update Total cases 3 017 766 12 879 Current cases 1 915 580 856 Deaths 207 722 468 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]




---



**Step4**: Create a dataset class
* Implement a custom TokenData class inheriting from torch.utils.data.Dataset.
* Use the train argument to toggle between training (train_tokens, train_Y) and testing datasets (test_tokens, test_Y).
* Define __len__ to return the dataset length and __getitem__ to provide tokenized inputs (input_ids, attention_mask) and labels (labels) as tensors.
* Ensure the class works correctly by retrieving a sample and verifying the output format.

In [14]:
class TokenData(Dataset):
    def __init__(self, train = False):
        if train:
            self.text_data = train_X
            self.tokens = train_tokens
            self.labels = list(train_Y)
        else:
            self.text_data = test_X
            self.tokens = test_tokens
            self.labels = list(test_Y)

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        sample = {}
        for k, v in self.tokens.items():
            sample[k] = torch.tensor(v[idx])
        sample['labels'] = torch.tensor(self.labels[idx])
        return sample

**Key Insights**: It ensures compatibility with DataLoader by enabling indexed access to tokenized inputs and their corresponding labels.



---



**Step5**: Create Dataloaders

* Initialize the `TokenData` class to create train_dataset and test_dataset objects for training and testing datasets.
* Set `batch_size=30` and enable shuffling for the training dataset to improve model generalization.
* Verify that train_loader and test_loader correctly return batches of tokenized inputs and labels.

In [25]:
batch_size = 300
#change to 30
train_dataset = TokenData(train = True)
test_dataset = TokenData(train = False)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)



---



**Step6:** Define Model
*  Use BertForSequenceClassification with bert-base-cased to load a pre-trained BERT model for sequence classification tasks.
* Use AdamW optimizer with a learning rate of 1e-5 to fine-tune the model parameters. This optimizer is well-suited for transformer-based models.
* Use CrossEntropyLoss to compute the loss for multi-class classification (as the dataset contains two classes).

In [26]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-cased') # Pre-trained model
optimizer = optim.AdamW(bert_model.parameters(), lr=1e-5) # Optimizer
loss_fn = nn.CrossEntropyLoss() # Loss function

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




---



* num_eochs = 3
* Use bert_model.to(device) to move the model to a GPU if available, or keep it on the CPU. This ensures efficient computation, especially for large datasets or models like BERT.

In [27]:
num_epochs = 1
# change to 3
bert_model.to(device) # Transfer model to GPU if available

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e



---



**Step7:** Model Training

In [ ]:
for epoch in range(num_epochs):
    bert_model.train()
    total_train_loss = 0.0  # Initialize total loss for each epoch

    for i, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        # Set gradients to zero
        optimizer.zero_grad()

        # Pass data to the model
        outputs = bert_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])

        # Get logits and calculate the loss
        pred = outputs.logits
        loss = loss_fn(pred, batch['labels'])

        # Backpropagation
        loss.backward()

        # Optimizing model parameters
        optimizer.step()

        # Accumulate the loss for the epoch
        total_train_loss += loss.item()
        print(i, total_train_loss)

    # Calculate and log the average loss for the epoch
    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Training Loss: {avg_train_loss:.4f}")




---



**Step8:** Model Evaluation

In [ ]:
# Set model to evaluation mode
bert_model.eval()

# Variables for tracking accuracy and loss
correct = 0
total = 0
total_test_loss = 0.0
all_preds = []
all_labels = []

# Disable gradient computation during testing
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = bert_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])

        # Get logits
        logits = outputs.logits

        # Calculate loss
        loss = loss_fn(logits, batch['labels'])
        total_test_loss += loss.item()

        # Get predictions
        preds = logits.argmax(dim=1)

        # Update the correct count and total number of samples
        correct += (preds == batch['labels']).sum().item()
        total += batch['labels'].size(0)

        # Store predictions and labels for confusion matrix
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate the average test loss
avg_test_loss = total_test_loss / len(test_loader)

# Calculate overall accuracy
accuracy = (correct / total) * 100

# Print overall results
print(f"Test Set - Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.2f}%")

Test Set - Loss: 0.2948, Accuracy: 88.53%


**BONUS**: Visualize the Confusion Matrix



---



**Step9**: Inference with new Tweets

* Run this inference code to predict the output for test_tweets

In [ ]:
# Test tweets
test_tweets = [
    "The number of COVID-19 cases has increased in the past week.",
    "Just watched an amazing sunset at the beach!",
    "New vaccination centers are opening up to combat the spread of coronavirus.",
    "A virus has impacted the global ecosystem a lot",
    "India lost the match againt Australia at Adelaide"
]

# Predict function for a single tweet
def predict_tweet(tweet_text):
    # Tokenize
    tokens = tokenizer(tweet_text, padding=True, truncation=True, return_tensors='pt')
    tokens = {k: v.to(device) for k, v in tokens.items()}

    # Predict
    bert_model.eval()
    with torch.no_grad():
        outputs = bert_model(**tokens)
        probs = torch.softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probs, dim=1)

    return {
        'text': tweet_text,
        'is_covid': bool(prediction.item()),
        'confidence': probs[0][prediction[0]].item() * 100
    }

# Make predictions for test tweets
print("Sample Tweet Predictions:\n")
for tweet in test_tweets:
    result = predict_tweet(tweet)
    print(f"Tweet: {result['text']}")
    print(f"Prediction: {'COVID-related' if result['is_covid'] else 'Not COVID-related'}")
    print(f"Confidence: {result['confidence']:.2f}%")
    print("-" * 50 + "\n")

Sample Tweet Predictions:

Tweet: The number of COVID-19 cases has increased in the past week.
Prediction: COVID-related
Confidence: 99.03%
--------------------------------------------------

Tweet: Just watched an amazing sunset at the beach!
Prediction: Not COVID-related
Confidence: 95.06%
--------------------------------------------------

Tweet: New vaccination centers are opening up to combat the spread of coronavirus.
Prediction: COVID-related
Confidence: 98.82%
--------------------------------------------------

Tweet: A virus has impacted the global ecosystem a lot
Prediction: COVID-related
Confidence: 98.08%
--------------------------------------------------

Tweet: India lost the match againt Australia at Adelaide
Prediction: Not COVID-related
Confidence: 83.30%
--------------------------------------------------



**Key Insights**:

 * Each tweet is tokenized into a format suitable for BERT using the tokenizer, ensuring padding and truncation for uniform input length. The tokenized inputs are moved to the appropriate device (GPU/CPU).

 * The pre-trained BERT model is used in evaluation mode (eval()), and predictions are obtained using `torch.softmax` to calculate class probabilities. The highest probability determines whether the tweet is COVID-related or not.

 * The result includes the predicted class (COVID-related or Not COVID-related) along with the confidence level, offering clarity and actionable insights from the model's output.



---

